### Historical Bikeshare Trip Data


In addition to the streaming data from the API, DC Capital Bikeshare provides monthly .csv files in an [S3 Bucket](https://s3.amazonaws.com/capitalbikeshare-data/index.htm). Here's our workflow for this notebook:

1. Download the .csv files 
2. Concatenate them into a master .csv 
3. Upload this to an S3 bucket of our own 
4. Query against this table using Athena
5. Explore trends in the data.

The idea behind aggregating the monthly csvs into a single file is to make querying all the historical data more straightforward, as well as to practice working with the `boto3` S3 SDK. I also published this dataset to [Kaggle](https://www.kaggle.com/datasets/alexsocarras/dc-capital-bikeshare). 

In [6]:
import requests
import boto3
import io
import zipfile
import csv
import pandas as pd
import re
import os
from bs4 import BeautifulSoup

##### Downloading the CSVs 

In [12]:
!mkdir data_temp

In [ ]:
bucket_name = 'capitalbikeshare-data'
prefix = ''

s3 = boto3.client('s3')

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

csv_content = ""
for obj in response['Contents']:
    
    if obj['Key'].endswith('.zip'):
        
        response = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
        zipfile_content = zipfile.ZipFile(io.BytesIO(response['Body'].read()))

        for filename in zipfile_content.namelist():

            if os.path.exists(f"./data_temp/{filename}"):
                continue

            csv_content = zipfile_content.read(filename)
         
            with open(f"./data_temp/{filename}", "wb") as fp:
                fp.write(csv_content)

In [ ]:
total_size = 0
for dirpath, dirnames, filenames in os.walk('./data_temp/'):
    for f in filenames:
        fp = os.path.join(dirpath, f)
        total_size += os.path.getsize(fp)

total_size # ~4.819 GB

5174216799

##### Concatenating CSVs

Before we concatenate we have to determine the different schemas used in each file. 

In [ ]:
file_metadata = []
for file in os.listdir("./data_temp"): 

    with open(os.path.join("./data_temp", file), "r") as fp:
        header = fp.readline().strip()
        header = re.sub('"',"", header)


    year = re.match('[0-9]{4}', file)[0]
    try:
        quarter = re.search(r'(?<=Q)[0-9]', file)[0]
    except: 
        quarter = "" 
    try: 
        month = re.search(r'(?<=\d{4})\d{2}', file)[0]
    except:
        month = "" 

    file_metadata.append({'year':year, 'month':month, 'quarter':quarter, 'header':header, 'filepath':'./data_temp/' + file})

df = pd.DataFrame(file_metadata)
df.sort_values(["year","month", "quarter"], inplace=True)
df

,year,month,quarter,header,filepath
28,2010,,,"Duration,Start date,End date,Start station num...",./data/2010-capitalbikeshare-tripdata.csv
85,2011,,,"Duration,Start date,End date,Start station num...",./data/2011-capitalbikeshare-tripdata.csv
10,2012,,1,"Duration,Start date,End date,Start station num...",./data/2012Q1-capitalbikeshare-tripdata.csv
0,2012,,2,"Duration,Start date,End date,Start station num...",./data/2012Q2-capitalbikeshare-tripdata.csv
87,2012,,3,"Duration,Start date,End date,Start station num...",./data/2012Q3-capitalbikeshare-tripdata.csv
...,...,...,...,...,...
18,2022,10,,"ride_id,rideable_type,started_at,ended_at,star...",./data/202210-capitalbikeshare-tripdata.csv
42,2022,11,,"ride_id,rideable_type,started_at,ended_at,star...",./data/202211-capitalbikeshare-tripdata.csv
3,2022,12,,"ride_id,rideable_type,started_at,ended_at,star...",./data/202212-capitalbikeshare-tripdata.csv
81,2023,01,,"ride_id,rideable_type,started_at,ended_at,star...",./data/202301-capitalbikeshare-tripdata.csv


In [ ]:
df.header.value_counts()


Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type                                             53
ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual    35
Name: header, dtype: int64

In [ ]:
new_header = "ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual"
df[df['header'] == new_header].iloc[0] # New header first used 04/2020

year                                                     2020
month                                                      04
quarter                                                      
header      ride_id,rideable_type,started_at,ended_at,star...
filepath          ./data/202004-capitalbikeshare-tripdata.csv
Name: 72, dtype: object

`rideable_type`, `start_lat`, `start_lng`, `end_lat`, and `end_lng` are notable new fields in the new schema format. Let's concatenate the files but keep them separate according to the different schema versions. 

In [ ]:
old_schema_files = list(df['filepath'][df['header'] != new_header])
new_schema_files = list(df['filepath'][df['header'] == new_header])

In [ ]:
import glob

file_gen = (pd.read_csv(file) for file in old_schema_files)

df = pd.concat(file_gen, ignore_index=True)
df.columns = [re.sub(" ","_", col.lower()) for col in df.columns]
df.to_csv("data/bks_tripdata_v1.csv", index=False)

In [ ]:
file_gen = (pd.read_csv(file) for file in new_schema_files)
df = pd.concat(file_gen, ignore_index=True)
df.to_csv("data/bks_tripdata_v2.csv", index=False)

/home/alex/portfolio/projects/alaska-etl/venv/lib/python3.7/site-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [ ]:
%%bash 

mv ./data_temp/bks*.csv ../data/

rm -rf ./data_temp

##### Upload to S3  

In [2]:
# s3 = boto3.client('s3')
# bucket_name = 'dc-bike-private' # Made this in Management Console
# file_path = '../data/bks_tripdata_v2.csv'
# object_name = 'dc-bike-private/bks_tripdata_v2.csv'
# 
# with open(file_path, "rb") as f:
    # s3.upload_fileobj(f, bucket_name, object_name)
# 
# print(f"File {file_path} uploaded to S3 bucket {bucket_name} with key {object_name}")

## As a function: 
def upload_file_s3(s3_client, file_path, bucket_name) -> None:

  object_name = os.path.basename(file_path)

  with open(file_path, "rb") as f:
    s3_client.upload_fileobj(f, bucket_name, object_name)

  print(f"File {file_path} uploaded to S3 bucket {bucket_name} with key {object_name}")

upload_file_s3(s3, '../data/bks_tripdata_v2.csv', 'dc-bike-private')

##### Querying the data with Athena 

After uploading the data to our S3 bucket, I created a database via the Athena console and connected it to our bucket folder `dc-bike-private`. Then I created a table based off the `bks_tripdata_v2.csv` file in our bucket with the following query in the query editor:

```sql 

CREATE EXTERNAL TABLE IF NOT EXISTS `dc_bike_private_db`.`bks_tripdata_v2_table` (
  `ride_id` string,
  `rideable_type` string,
  `started_at` timestamp,
  `ended_at` timestamp,
  `start_station_name` string,
  `start_station_id` int,
  `end_station_name` string,
  `end_station_id` float,
  `start_lat` float,
  `start_lng` float,
  `end_lat` float,
  `end_lng` float,
  `member_casual` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://dc-bike-private/bks_tripdata_v2/'
TBLPROPERTIES ('classification' = 'csv');
```

Note that when you create a table in Athena from data in an S3 bucket, you have to connect to a folder and not directly to a file. Having a folder with multiple files in it can cause serious parsing issues. So you should create a separate folder in your bucket to individually house each file you want to connect.  

We can now query against this table using the boto3 SDK.


In [16]:
import time 

BUCKET_ID = "dc-bike-private"
DB_ID = "dc_bike_private_db" 
TABLE_ID = "bks_tripdata_v2_table"

athena_client = boto3.client('athena')

def execute_query(query, database, output_location=f's3://{BUCKET_ID}/query-results/'):
    
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database},
        ResultConfiguration={'OutputLocation': output_location}
    )

    query_execution_id = response['QueryExecutionId']
    state = None

    while state not in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        state = response['QueryExecution']['Status']['State']
        time.sleep(1)

    if state == 'FAILED':
        raise Exception(response['QueryExecution']['Status']['StateChangeReason'])
    
    result_set = []
    column_names = []

    results = athena_client.get_query_results(QueryExecutionId=query_execution_id)
    for row in results['ResultSet']['Rows']:
        if not column_names:
            column_names = [col['VarCharValue'] for col in row['Data']]
        else:
            result_set.append({column_names[i]: row['Data'][i]['VarCharValue'] for i in range(len(column_names))})

    return result_set

In [20]:
query = f"""SELECT * FROM {TABLE_ID} LIMIT 1"""

execute_query(query, DB_ID)

[{'ride_id': 'D3B94F4CE7AC46BC',
  'rideable_type': 'docked_bike',
  'started_at': '2020-05-25 16:35:25.000',
  'ended_at': '2020-05-25 16:48:49.000',
  'start_station_name': 'Maine Ave & 7th St SW',
  'start_station_id': '31609',
  'end_station_name': 'Maine Ave & 9th St SW',
  'end_station_id': '31646.0',
  'start_lat': '38.878693',
  'start_lng': '-77.023056',
  'end_lat': '38.88044',
  'end_lng': '-77.02524',
  'member_casual': 'casual'}]